In [2]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import nltk

import csv
import io
import string

from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))

global str

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors

from translate import Translator


from time import time 
from collections import defaultdict

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
file=pd.read_csv('USelectionw2v.csv')


In [4]:
file.head()

,created_at,full_text,hashtags,user_mentions
0,Sat Jun 13 20:41:42 +0000 2020,@traceybra @TheLondonHughes #AllLivesMatter,AllLivesMatter,"traceybra,TheLondonHughes"
1,Sat Jun 13 20:41:42 +0000 2020,RT @beardedfinance: You look like an EasyJet f...,NaN,beardedfinance
2,Sat Jun 13 20:41:42 +0000 2020,RT @larryelder: #alllivesmatter https://t.co/A...,alllivesmatter,larryelder
3,Sat Jun 13 20:41:42 +0000 2020,RT @EWindt: Lieve @NUnl #AllLivesMatter #AllL...,"AllLivesMatter,AllLivesMatter,AllLivesMatter,A...","EWindt,NUnl"
4,Sat Jun 13 20:41:43 +0000 2020,@realDonaldTrump Congrats to the graduates. Bu...,NaN,realDonaldTrump


In [5]:
file.full_text.count()

102629

In [6]:
file_cleaned = file.drop(columns=['created_at','user_mentions']).reset_index(drop=True)
file_cleaned['tweet']=file['full_text']

In [7]:
#file_cleaned = file_cleaned.dropna().reset_index(drop= True)
file_cleaned = file_cleaned.drop_duplicates().reset_index(drop= True)


In [8]:
file_cleaned.full_text.count()


37237

In [9]:
file_cleaned['hashtags']=file_cleaned['hashtags'].str.lower()


In [10]:
file_cleaned.head()

,full_text,hashtags,tweet
0,@traceybra @TheLondonHughes #AllLivesMatter,alllivesmatter,@traceybra @TheLondonHughes #AllLivesMatter
1,RT @beardedfinance: You look like an EasyJet f...,NaN,RT @beardedfinance: You look like an EasyJet f...
2,RT @larryelder: #alllivesmatter https://t.co/A...,alllivesmatter,RT @larryelder: #alllivesmatter https://t.co/A...
3,RT @EWindt: Lieve @NUnl #AllLivesMatter #AllL...,"alllivesmatter,alllivesmatter,alllivesmatter,a...",RT @EWindt: Lieve @NUnl #AllLivesMatter #AllL...
4,@realDonaldTrump Congrats to the graduates. Bu...,NaN,@realDonaldTrump Congrats to the graduates. Bu...


In [11]:
def text_to_word_list(text, process):
    ''' Pre process and convert texts to a list of words 
    method inspired by method from eliorc github repo: https://github.com/eliorc/Medium/blob/master/MaLSTM.ipynb'''
    text = process(text)
    text = text.lower()
    # Clean the text
    text = sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)(?:(?:\/[^\s/]*))*'," ",text)
        

    text= sub(r' (@[a-zA-Z]\S+ )'," ",text)
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r"^rt ", "", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ", text)
    text = sub(r"\?", " ", text)
    text = sub(r" [1-9]* ",' ',text)
    text = sub(r"'", " ", text)
    text = sub(r":", " ", text)
    text = sub(r"\s{2,}", " ", text)
    
    text = text.split()

    return text

In [12]:
file_cleaned.full_text = file_cleaned.full_text.apply(lambda x: text_to_word_list(x, unidecode))


In [15]:
file_cleaned.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37237 entries, 0 to 37236
Data columns (total 3 columns):
full_text    37237 non-null object
hashtags     21340 non-null object
tweet        37237 non-null object
dtypes: object(3)
memory usage: 872.9+ KB


In [13]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [14]:
file_cleaned['full_text']=file_cleaned['full_text'].apply(lambda x:[ lemmatizer.lemmatize(y) for y in x])

In [15]:
#remove negative words from stopwords
import re
l1=[]
l2=[]
l3=['no','nor','couldn','hasn','hadn','haven','didn','but']
for i in stop_words:
    str=''
    x=re.findall('(.*n.t$)',i)
    if len(x)!=0:
        a=str.join(x)
        l2.append(a)

for i in stop_words:
    if not i in l2:
        if i not in l3:
            l1.append(i)
#l1 is list of stopwords without negative words


In [16]:
file_cleaned.full_text=file_cleaned['full_text'].apply(lambda x:[item for item in x if item not in l1])

In [17]:
file_model = file_cleaned.copy()
file_model = file_model[file_model.full_text.str.len()>1]


In [18]:
sent = [row for row in file_model.full_text]
phrases = Phrases(sent, min_count=5, progress_per=10000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]


INFO - 18:44:57: collecting all words and their counts
INFO - 18:44:57: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 18:44:57: PROGRESS: at sentence #10000, processed 95706 words and 79141 word types
INFO - 18:44:57: PROGRESS: at sentence #20000, processed 183067 words and 137004 word types
INFO - 18:44:57: PROGRESS: at sentence #30000, processed 270677 words and 194752 word types
INFO - 18:44:57: collected 229426 word types from a corpus of 320849 words (unigram + bigrams) and 35261 sentences
INFO - 18:44:57: using 229426 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>
INFO - 18:44:57: source_vocab length 229426
INFO - 18:44:59: Phraser built with 2034 phrasegrams


['look_like', 'easyjet_flight', 'attendant']

In [19]:
w2v_model = Word2Vec(min_count=10,
                     window=5,
                     size=300,
                     sample=5e-5, 
                     alpha=0.04, 
                     min_alpha=0.0007, 
                     negative=3,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(sentences, progress_per=30000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

INFO - 18:45:00: collecting all words and their counts
INFO - 18:45:00: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 18:45:00: PROGRESS: at sentence #30000, processed 249106 words, keeping 39829 word types
INFO - 18:45:00: collected 45226 word types from a corpus of 295656 raw words and 35261 sentences
INFO - 18:45:00: Loading a fresh vocabulary
INFO - 18:45:00: effective_min_count=10 retains 4529 unique words (10% of original 45226, drops 40697)
INFO - 18:45:00: effective_min_count=10 leaves 214708 word corpus (72% of original 295656, drops 80948)
INFO - 18:45:01: deleting the raw counts dictionary of 45226 items
INFO - 18:45:01: sample=5e-05 downsamples 1559 most-common words
INFO - 18:45:01: downsampling leaves estimated 109377 word corpus (50.9% of prior 214708)
INFO - 18:45:01: estimated required memory for 4529 words and 300 dimensions: 13134100 bytes
INFO - 18:45:01: resetting layer weights


Time to build vocab: 0.01 mins


In [20]:
start = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=60, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

INFO - 18:45:03: training model with 7 workers on 4529 vocabulary and 300 features, using sg=0 hs=0 sample=5e-05 negative=3 window=5
INFO - 18:45:04: worker thread finished; awaiting finish of 6 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 4 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:04: EPOCH - 1 : training on 295656 raw words (109358 effective words) took 0.7s, 146317 effective words/s
INFO - 18:45:04: worker thread finished; awaiting finish of 6 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:04: worker thread finished; awaiting finish of 4 more thread

INFO - 18:45:14: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:14: EPOCH - 13 : training on 295656 raw words (109109 effective words) took 0.9s, 127279 effective words/s
INFO - 18:45:14: worker thread finished; awaiting finish of 6 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 4 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:14: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:14: EPOCH - 14 : training on 295656 raw words (109197 effective words) took 0.8s, 134005 effective words/s
INFO - 18:45:15: worker thread finished; awaiting finish of 6 more threads
INFO - 18:

INFO - 18:45:25: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:25: EPOCH - 26 : training on 295656 raw words (109453 effective words) took 0.9s, 121487 effective words/s
INFO - 18:45:25: worker thread finished; awaiting finish of 6 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 4 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:25: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:25: EPOCH - 27 : training on 295656 raw wo

INFO - 18:45:36: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:36: worker thread finished; awaiting finish of 4 more threads
INFO - 18:45:36: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:36: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:36: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:36: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:36: EPOCH - 39 : training on 295656 raw words (109397 effective words) took 0.9s, 122914 effective words/s
INFO - 18:45:37: worker thread finished; awaiting finish of 6 more threads
INFO - 18:45:37: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:37: worker thread finished; awaiting finish of 4 more threads
INFO - 18:45:37: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:37: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:37: worker thread finished; awaiting finis

INFO - 18:45:47: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:47: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:47: EPOCH - 51 : training on 295656 raw words (109647 effective words) took 0.9s, 127772 effective words/s
INFO - 18:45:48: worker thread finished; awaiting finish of 6 more threads
INFO - 18:45:48: worker thread finished; awaiting finish of 5 more threads
INFO - 18:45:48: worker thread finished; awaiting finish of 4 more threads
INFO - 18:45:48: worker thread finished; awaiting finish of 3 more threads
INFO - 18:45:48: worker thread finished; awaiting finish of 2 more threads
INFO - 18:45:48: worker thread finished; awaiting finish of 1 more threads
INFO - 18:45:48: worker thread finished; awaiting finish of 0 more threads
INFO - 18:45:48: EPOCH - 52 : training on 295656 raw words (109199 effective words) took 0.8s, 132163 effective words/s
INFO - 18:45:49: worker thread finished; awaiting finish of 6 more threads
INFO - 18:

Time to train the model: 0.88 mins


In [21]:

w2v_model.save("word2vec.model")

INFO - 18:48:05: saving Word2Vec object under word2vec.model, separately None
INFO - 18:48:05: not storing attribute vectors_norm
INFO - 18:48:05: not storing attribute cum_table
INFO - 18:48:05: saved word2vec.model


In [22]:
w2v_model.wv.most_similar(positive=["blm"])

[('mahyartousi', 0.7375154495239258),
 ('blacklivesmatter', 0.6665360927581787),
 ('blacklivesmatteruk', 0.6616393327713013),
 ('londonprotests', 0.641721248626709),
 ('savage', 0.6058285236358643),
 ('peacefulprotest', 0.5977144241333008),
 ('throat', 0.5964519381523132),
 ('alllivesmatter', 0.5962355136871338),
 ('whitelivesmatter_alllivesmatter', 0.5953236222267151),
 ('alllivesmatter_bluelivesmatter', 0.5834579467773438)]

In [28]:
w2v_model.wv.most_similar(positive=["trump"])

[('donaldtrump', 0.6288428902626038),
 ('trumpdictatorship', 0.5091039538383484),
 ('potus', 0.4876251220703125),
 ('donaldjtrump', 0.4774830937385559),
 ('usa', 0.4763261079788208),
 ('realdonaldtrump', 0.4703081250190735),
 ('donald_trump', 0.4647345542907715),
 ('gold', 0.4611928462982178),
 ('putin', 0.4549914598464966),
 ('bunker', 0.4526779055595398)]

In [29]:
w2v_model.wv.similarity("donaldtrump", 'trump')

0.62884283

In [32]:
file_export = file_model.copy()
file_export.full_text = file_export.full_text.apply(lambda x: ' '.join(bigram[x]))
file_export['Tweet']=file_cleaned['tweet']
file_export['hashtags']=file_cleaned['hashtags']

In [33]:
file_export.to_csv('cleaned_datasets.csv', index=False)

#example


In [174]:
import re
strng= "rt njkhhu @realDonaldTrump Youâ€™re 3rd notworshippers9rt  jhyy 45 are NOT in the majority.â€¦ https://t.co/49lM1eO78J"

In [175]:
strng=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*'," ",strng)
strng=re.sub(r' +@[a-zA-Z]\S+ '," ",strng)
strng = re.sub(r' [0-9]* '," ",strng)
strng= re.sub(r'^rt ',"",strng)

In [176]:
print(strng)

njkhhu Youâ€™re 3rd notworshippers9rt jhyy are NOT in the majority.â€¦ 


In [168]:
import re
thestring = "text1\ntext2\nhttp://url.com/bla1/blah1/\ntext3\ntext4\nhttp://url.com/bla2/blah2/\ntext5\ntext6"

URLless_string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', thestring)
print (URLless_string)

text1
text2

text3
text4

text5
text6


In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PRIYANKA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [213]:
strng= "rt njkhhu @realDonaldTrump Youâ€™re better 3rd notworshippers9rt studies jhyy 45 are NOT in the majority.â€¦ https://t.co/49lM1eO78J"

In [187]:

from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

print(lemmatizer.lemmatize(strng))
  
print("rocks :", lemmatizer.lemmatize("rocks")) 
print("corpora :", lemmatizer.lemmatize("corpora")) 
  
# a denotes adjective in "pos" 
print("better :", lemmatizer.lemmatize("better", pos ="a"))

rt njkhhu @realDonaldTrump Youâ€™re 3rd notworshippers9rt studies jhyy 45 are NOT in the majority.â€¦ https://t.co/49lM1eO78J
rocks : rock
corpora : corpus
better : good


In [214]:
for word in strng.split():
    print(lemmatizer.lemmatize(word))

rt
njkhhu
@realDonaldTrump
Youâ€™re
better
3rd
notworshippers9rt
study
jhyy
45
are
NOT
in
the
majority.â€¦
https://t.co/49lM1eO78J


In [64]:
lemmatizer = WordNetLemmatizer()
for w in strng.split( ):
    w=strng.replace(w,lemmatizer.lemmatize(w))
print(w)
    #print("Lemma for {} is {}".format(w, lemmatizer.lemmatize(w)))


rt njkhhu @realDonaldTrump Youâ€™re 3rd notworshippers9rt studies jhyy 45 are NOT in the majority.â€¦ https://t.co/49lM1eO78J


In [94]:

	wordnet_lemmatizer = WordNetLemmatizer()
	text = "studies studying cries cry"
	tokenization = nltk.word_tokenize(text)
	 for w in tokenization:
		print("Lemma for {} is {}".format(w, wordnet_lemmatizer.lemmatize(w)))  

IndentationError: unexpected indent (<ipython-input-94-df413601eb95>, line 2)

In [13]:
def get_stemmed_text(corpus):
    from nltk.stem.porter import PorterStemmer
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]

stemmed_reviews = get_stemmed_text(reviews_train_clean)

NameError: name 'reviews_train_clean' is not defined

In [14]:
def get_lemmatized_text(corpus):
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

lemmatized_reviews = get_lemmatized_text(reviews_train_clean)

NameError: name 'reviews_train_clean' is not defined

In [15]:
result = translator.translate('Abramos hilo de lo que sucediÃ³ esta madrugada para los que apenas se acaban de despertar ')

In [16]:
print(result.text)

AttributeError: 'str' object has no attribute 'text'

In [29]:
#correct
from translate import Translator
data = ['Dobrý deň', 'majestátny orol', 'krehká dohoda']
translator= Translator(to_lang="English")

for word in data:
    translated = translator.translate(word)
    print(translated)

Dobrý deň
majestátny orol
krehká dohoda


In [ ]:
df['stemmed'] = df['unstemmed'].apply(lambda x: [stemmer.stem(y) for y in x]) # Stem every word.
df = df.drop(columns=['unstemmed'])